In [34]:
#r "nuget:SharpVG"
open SharpVG    

type Pos = { 
    x: int 
    y: int 
}
module Pos = 
    let rotate90 (pos:Pos):Pos =
        { pos with x = (pos.y * -1); y = pos.x}

    let rotate270 (pos:Pos):Pos =
        { pos with x = pos.y; y = (pos.x * -1) }
type Range = {
    distance: int
    top: Pos
    bottom: Pos
    left: Pos
    right: Pos
}
module Range =
    let rotate r (range:Range) =
        { range with
            top = r range.left
            bottom = r range.right
            left = r range.bottom
            right = r range.top
        }
type Sensor = { 
    pos: Pos
    beacon: Pos
    range: Range
}
module Sensor =
    open System.Text.RegularExpressions

    let rotate r (sensor:Sensor) =
        { sensor with 
            pos = r sensor.pos
            beacon = r sensor.beacon
            range = Range.rotate r sensor.range
        }

    let regex = 
        Regex("Sensor at x=(?<sx>-?\d+), y=(?<sy>-?\d+): closest beacon is at x=(?<bx>-?\d+), y=(?<by>-?\d+)")

    let range (sensor:Pos) (beacon: Pos) =
        let distance =
            Math.Abs(sensor.x - beacon.x) +
            Math.Abs(sensor.y - beacon.y)

        let top = { sensor with y = sensor.y - distance }
        let bottom = { sensor with y = sensor.y + distance }
        let left = { sensor with x = sensor.x - distance }
        let right = { sensor with x = sensor.x + distance }
        
        top, bottom, left, right, distance

    let parse line =
        regex.Matches(line)
        |> Seq.map (fun m -> 
            let sx = m.Groups.["sx"].Value |> int
            let sy = m.Groups.["sy"].Value |> int
            let bx = m.Groups.["bx"].Value |> int
            let by = m.Groups.["by"].Value |> int
            let sensor = { x = sx; y = sy } 
            let beacon = { x = bx; y = by } 
            let top, bottom, left, right, distance = range sensor beacon
            { 
                pos = sensor
                beacon = beacon
                range = { 
                    top = top
                    left = left
                    bottom = bottom
                    right = right
                    distance = distance 
                } 
            }
        )
        |> Seq.head
    
    let beaconsAt line (sensors:Sensor seq) =
        sensors
        |> Seq.filter (fun it -> it.beacon.y = line )
        |> Seq.map (fun it -> it.beacon)
        |> Set.ofSeq
        |> Seq.length

type Slice =
| Start of int
| End of int
module Slice =
    let intersect line sensors =
        let merge slices (sa:Slice, ea:Slice) =
            [ sa; ea ]
            |> Seq.append slices

        let downFun (pos:Pos) =
            let b = pos.y - pos.x
            fun x -> x - b

        let riseFun (pos:Pos) =
            let b = pos.y + pos.x
            fun x -> b - x

        let i_aux line (sensor:Sensor) =
            if line >= sensor.range.top.y && line < sensor.pos.y then 
                let s = (riseFun sensor.range.top) line
                let e = (downFun sensor.range.top) line
                Some (Start s, End e)
            elif line >= sensor.pos.y && line <= sensor.range.bottom.y then
                let s = (downFun sensor.range.bottom) line
                let e = (riseFun sensor.range.bottom) line
                Some (Start s, End e)
            else
                None

        let compact slices =
            let rec c_aux stack acc slices =
                match slices with 
                | [] -> acc
                | head :: tail ->
                    match head with
                    | Start s when stack = 0 -> c_aux (stack + 1) ([head] |> Seq.append acc) tail
                    | Start _ -> c_aux (stack + 1) acc tail
                    | End e when stack = 1 -> c_aux 0 ([head] |> Seq.append acc) tail
                    | End _ -> c_aux (stack - 1) acc tail
            c_aux 0 List.empty (slices |> List.ofSeq)

        sensors
        |> Seq.map (i_aux line)
        |> Seq.choose id
        |> Seq.fold merge Seq.empty
        |> Seq.sortBy (fun it -> match it with Start s -> s | End e -> e )
        |> compact

    let gaps (slices:Slice seq) = 
        slices
        |> Seq.map(fun it -> match it with Slice.Start s -> s | Slice.End e -> e)
        |> Seq.chunkBySize 2
        |> Seq.pairwise
        |> Seq.map(fun (a,b) -> b.[0] - a.[1])
        |> Seq.filter(fun it -> it > 1 )
    
    let findGap (range:int seq) (sensors:Sensor seq) =
        range
        |> Seq.map (fun line -> async { return sensors |> intersect line |> gaps } )
        |> Async.Parallel
        |> Async.RunSynchronously
        |> Seq.tryFindIndex (Seq.isEmpty >> not)
    
    let sum slices =
        let rec s_aux slices acc =
            match slices with
            | [] -> acc
            | head :: tail ->
                match head with 
                | Start s -> s_aux tail (acc - s)
                | End e -> s_aux tail (acc + e)
        s_aux (slices|>List.ofSeq) 0

Installed Packages SharpVG, 0.0.16

In [40]:
module Preview =
    open SharpVG    
    
    let showSvg svg = display(HTML(Svg.toString svg))

    let point (p:Pos) = (p.x ,p.y ) |> Point.ofInts

    let dimensions (sensors:Sensor seq) =
        let pt p =  (p.x ,p.y)
        let points =
            sensors
            |> Seq.map (fun it -> seq { 
                yield pt it.pos
                yield pt it.beacon
                yield pt it.range.top  
                yield pt it.range.bottom  
                yield pt it.range.left  
                yield pt it.range.right
            })
            |> Seq.concat
        let top, bottom =
            let lines = points |> Seq.map snd |> Seq.sort
            lines |> Seq.head, lines |> Seq.last
        let left, right =
            let columns = points |> Seq.map fst |> Seq.sort
            columns |> Seq.head, columns |> Seq.last

        (Point.ofInts(left, top)), (Point.ofInts(right, bottom))

    let sensor (s:Sensor) =
        let centre = point s.pos
        let style = Style.create (Name Colors.Blue) (Name Colors.Blue) (Length.ofInt 0) 1.0 1.0
        let radius = (s.range.distance / 10) + 1
        Circle.create centre (Length.ofInt radius)
        |> Element.createWithStyle style

    let beacon (s:Sensor) =
        let centre = point s.beacon
        let style = Style.create (Name Colors.Red) (Name Colors.Red) (Length.ofInt 0) 0.5 0.5
        let radius = (s.range.distance / 10) + 1
        Circle.create centre (Length.ofInt radius)
        |> Element.createWithStyle style


    let range (s:Sensor) =
        let points =
            seq {
                yield point s.range.top
                yield point s.range.left
                yield point s.range.bottom 
                yield point s.range.right          
            }
        let style = Style.create (Name Colors.Green) (Name Colors.Red) (Length.ofPercent 0.05) 0.25 0.25

        points
        |> Polygon.ofSeq
        |> Element.createWithStyle style
    
    let plotSensor (s:Sensor) =
        seq {
            yield sensor s
            yield beacon s
            yield range s
        }

    let slices line (s: Slice seq) =
        let style = Style.create (Name Colors.Red) (Name Colors.Red) (Length.ofPercent 0.1) 1 1
        s 
        |> List.ofSeq
        |> List.map(fun it -> match it with Slice.Start s -> s | Slice.End e -> e )
        |> List.chunkBySize 2
        |> List.map(fun it -> Point.ofInts(it.[0], line), Point.ofInts(it.[1], line))
        |> List.map (fun (s,e) -> Line.create s e |> Element.createWithStyle style )


    let plot (topLeft:int*int) (bottomRight:int*int) line (sensors:Sensor seq) =
        //let tl, br = dimensions sensors
        let tl = Point.ofInts topLeft 
        let br = Point.ofInts bottomRight
        //display $"{tl} {br}"
        let area = Area.fromPoints tl br
        let viewbox = ViewBox.create tl area

        let slice_el =
            sensors 
            |> Slice.intersect line
            |> slices line

        let sensor_el =
            sensors
            |> Seq.map plotSensor |> Seq.concat
        
        slice_el |> Seq.append sensor_el    
        |> Svg.ofSeq
        |> Svg.withViewBox viewbox
        |> showSvg
            

In [35]:
let ResolutionFolder = __SOURCE_DIRECTORY__

let solve input line =
    
    let sensors =
        File.ReadLines( ResolutionFolder + input)
        |> Seq.map Sensor.parse

    let intersect =
        sensors
        |> Slice.intersect line

    let sum = intersect |> Slice.sum    

    let found = sensors |> Sensor.beaconsAt line

    display (sum - found)

    sensors

let sensors = solve "/input15.txt" 2000000

4961646

In [36]:
// WARNING This takes 5+ minutes to compute on my Core i7 10750H @2.6GHz
let y = sensors |> Slice.findGap [0 .. 4000000] |> Option.defaultValue 0

In [37]:
let rotated = sensors |> Seq.map (Sensor.rotate Pos.rotate90)

// WARNING This takes 5+ minutes to compute on my Core i7 10750H @2.6GHz
let x = rotated |> Slice.findGap [0 .. 4000000] |> Option.defaultValue 0

In [44]:
( (int64 x) * 4000000L ) + ( int64 y)

12274327017867

In [41]:
rotated |> Preview.plot (-4000000,0) (0,4000000) x

In [42]:
sensors |> Preview.plot (0,0) (4000000, 4000000) y